In [ ]:
import flamp
import numpy as np
import numpy.linalg as lin
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import sparse
from tqdm import tqdm

import matrix_functions as mf
from fa_performance import fa_performance
from paper_plots import plot_convergence_curves

flamp.set_dps(350)  # compute with this many decimal digits precision


In [ ]:
dim = 100
kappa = flamp.gmpy2.mpfr(1_000.)
lambda_min = flamp.gmpy2.mpfr(1.)
lambda_max = kappa * lambda_min

# a_diag = mf.flipped_model_spectrum(dim, kappa, 50, lambda_1=lambda_min)
# a_diag = mf.two_cluster_spectrum(dim, kappa, low_cluster_size=3, lambda_1=lambda_min)
# a_diag = mf.two_cluster_spectrum(dim, kappa, low_cluster_size=1, high_cluster_width=0.25, lambda_1=lambda_min)
# a_diag = mf.utils.linspace(lambda_min, lambda_max, num=dim)
# a_diag = mf.model_spectrum(dim, kappa, .9, lambda_1=lambda_min)
# a_diag = mf.two_cluster_spectrum(dim, kappa, low_cluster_width=0.001, high_cluster_width=0.5, low_cluster_size=5, lambda_1=lambda_min)
a_diag = mf.geometric_spectrum(dim, kappa, rho=1e-5, lambda_1=1)


plt.plot(a_diag)

In [ ]:
denom_deg = 2
def f(x):
    return x**(-denom_deg)

mu = a_diag[:-1] + np.diff(a_diag) / 2
# mu = a_diag[1:] - np.diff(a_diag).min()/2
b = mf.start_vec(a_diag, mu)

# b = flamp.ones(dim)
# b = flamp.to_mp(np.flip(mf.model_spectrum(dim, kappa, 0.9)))

# size of Krylov subspace
ks = list(range(1, dim, 10))  # + [dim]


In [ ]:
results = fa_performance(f, a_diag, b, ks, denom_degree=denom_deg, relative_error=True)
plot_convergence_curves(results, "Relative Error", "Number of matrix-vector products ($k$)", title=None, ax=None)
plt.savefig('output/convergence.svg')

# Detritus

In [ ]:
# import baryrat

# running_best_rational_bound = np.inf
# best_rational_bounds = []
# for k in range(99):
#     if k >= 2:
#         ##### WARNING! DON"T HARD CODE THE FUNCTION OK!
#         # aaa_interpolant = baryrat.aaa(np.array(spectrum_discritization, float), np.sqrt, mmax=(k//2))
#         # error = mf.norm(aaa_interpolant(spectrum_discritization) - np.array(f_spectrum_discritization, float), ord=np.inf)
#         # deg_k_error = (kappa ** (k//2)) * error
#         inv_sqrt = tyler_inv_sqrt((k//2), spectrum_discritization.min(), spectrum_discritization.max())
#         zolotarev_interpolant = lambda x: 1./inv_sqrt(x)
#     else:
#         deg_k_error = np.inf

#     running_best_rational_bound = min(running_best_rational_bound, deg_k_error)
#     best_rational_bounds.append(running_best_rational_bound)

# results['rational'] = np.array([min(best_rational_bound[:(k+1)]) for k in ks], dtype=float)

In [ ]:
# rational_bound_all_q = []
# for q in range(2,20):
#     rq_inv_sqrt = tyler_inv_sqrt(q, lambda_min, lambda_max)
#     rational_bound_q = []
#     for k in ks:
#         rational_approx_error = mf.norm(f(spectrum_discritization) - 1./rq_inv_sqrt(spectrum_discritization), ord=np.inf)
#         _, residual = flamp.qr_solve(krylov_basis[:, :k], mf.diagonal_fa(rq_inv_sqrt, a_diag, x), res=True) # uh-oh, this is 2-norm optimal, not A-norm optimal
#         krylov_error = mf.norm(residual, ord=np.inf)
#         rational_bound_q.append(rational_approx_error + (kappa ** q) * krylov_error)
#     rational_bound_all_q.append(rational_bound_q)

# results['other'] = np.array(rational_bound_all_q, float).min(axis=0)